仓位管理

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import statsmodels.api as sm
import QUANTAXIS as QA
import talib as ta
import datetime #, time

每次下注10元

In [2]:
result = []

for i in range(1000):
    
    init_assets = 1000
    assets = init_assets
    for j in range(1000):
        num = 10
        _result = np.random.choice([0.7, -1],p=[0.6, 0.4])
        assets += num * _result
        
    result.append(assets)
        
np.mean(result)

1198.538

每次下注总金额的1%

In [3]:
result = []

for i in range(1000):
    
    init_assets = 1000
    assets = init_assets
    for j in range(1000):
        num = assets * 0.01
        _result = np.random.choice([0.7, -1],p=[0.6, 0.4])
        assets += num * _result
        
    result.append(assets)
        
np.mean(result)

1235.2926791836294

利用凯利公式下注

f* = (b\*p - q) / b

通过凯利公式，当最近的胜率很低，应该减仓，
动态仓位管理，是决定性作用之一

In [8]:
b = 0.7
p = 0.6
q = 0.4

f_star = (b*p - q) / b

result = []

for i in range(1000):
    
    init_assets = 1000
    assets = init_assets
    for j in range(1000):
        num = assets * f_star
        _result = np.random.choice([0.7, -1],p=[0.6, 0.4])
        assets += num * _result
        
    result.append(assets)
        
np.mean(result)

1821.359345806627

仓位管理

```
仓位 = 投资资金/总资金

不同阶段: 建仓、加仓/减仓、清仓
资金分配：头寸规模的确定和分配
```

```
常见的加仓方式

1. 均匀加仓
2. 金字塔式加仓
3. 倒金字塔式加仓
```

确定头寸规模的四种模型

```
- 每一固定金额交易一个单位
- 等价值交易单位
- 百分比风险模型
- 百分比波动幅度模型
```

在展开下面的详细描述之前，哪一种加仓或者确定的头寸的方法更接近凯利公式?

如何确定每次下注的赔率，胜率是多少，不能用回测的结果去做修正，这个是未来函数!!

```
model 1 : 每一固定金额一个单位

model 2 : 等价值交易单位
```

```
model 3 : 百分比风险模型

CPR: Pos = cash / risk
各个交易品种和个股的risk是不一样的
每次交易允许承担的最大风险占总资金的比例，
及每个投资标的可接受的最大损失，上述中的risk，折算出可建立的头寸单位个数
个股承担的风险之和不能超过总风险
```

```
model 4 : 百分比波动幅度模型

根据每次交易允许承担的最大风险占总资金的比例，
以及每个投资标的在一段时间内的价格波动的幅度，折算出可建立头寸的单位个数
```

仓位分配的基本参数

```
是否考虑复利， 不懂
资金占比上限，单股的资金占比上限？还是所有股票的？还是两种都有？
单日分配资金上限，就像不可能把所有钱在一只股票上面花完，也不可能在一天花完
同时持股数上限
单只股票仓位上限
... ...
```

工程化实现的考虑

- 可用资金不足时的处理
    - 限制新的购买，忽略该信号
    - 在增加新股票之前，消除业绩最差的头寸
    - 为继续购买新的，将头寸规模确定的小一点
- 入选组合的优先级
    - 根据某种指标排序
    - 根据历史表现筛选

止盈和止损

```
止盈的意义
    - 火鸡的故事
    - 当风险逐渐变大时，落袋为安
    
 止损的意义
     - 鳄鱼法则
     - 留得青山在，不怕没柴烧
```

重要参考指标 - 平均波动率

- 平均波动率：过去N天波动率的平均值

- 波动率：max(H-L, abs(pre_close - H), abs(pre_close - L))

- 波动率反映了金融资产的风险水平

四种止盈方法
```
1. 目标止盈法
2. 回撤止盈
3. 大幅波动止盈
4. 技术信号止盈
```
多种方法相结合一定好么，为什么好，为什么不好

就像技术指标越多越复杂好么，如果不好，为什么

四种止损方法
```
1. 最大亏损法
2. 波动率倍数止损
3. 波动率标准差止损
4. 技术信号止损
```

首先复习一下成功的交易系统核心要素是什么

1. 赔率
2. 胜率
3. 交易成本
4. 交易频率
5. 头寸管理
6. 本金

交易系统核心要素

1. 市场：买卖什么
2. 头寸规模：买卖多少
3. 入市：什么时候买进
4. 止损：什么时候放弃一个头寸
5. 止盈：什么时候退出一个盈利的头寸
6. 战术：怎么买卖

基准策略和测试环境
```
以下可以看做一个策略的基本模板
```

- 股票池候选：上证50成分股，不调仓
- 双均线策略
    - 日K级别，MA10 vs MA30
    - 开盘前检测信号，金叉买，死叉麦
- 头寸规模确定
    - 均仓方案(等额资金分配)
- 测试环境
    - 回测区间：2016.1.1~2018.6.30，资金：1000万
    - programming language：python,平台:聚宽

In [ ]:
from jqdata import *

In [1]:
MA_WIN_1 = 10
MA_WIN_2 = 30

In [ ]:
def initialize(context):
    set_benchmark('000300.XSHG')
    set_option('use_real_price', True)
    # log.set_level('order', 'error')

按盈利比例均匀加仓

- 记录每只持仓股最后一次的买入价

- 如以当日开盘价相对于前一次买入价的盈利比例超过某个阈值，则等额加一次仓
    - 前提是当日没有任何止损或其它买入信号发生

改进方向

- 加入大盘趋势判断，是否触发浮动止盈?
- 将止盈止损的方法改变为按波动率倍数止损
- 分析几种优化方案，如何取舍，比如怎么让利润奔跑，仓位图更加饱满的情况，在行情特别不好的时候，及时退出，不过过高仓位，是否能及时退出呢，比如在模拟盘或者实盘中

完成第8课的预习

自己搭建一个系统

在quantaxis复现上述策略

首先在基准策略是上添加
 - 均匀加仓
 - 然后在均匀加仓的版本上修改，按波动率分配资金，并且加仓也按照这个方法来
 - 波动率上升，但回撤并未下降
 - 针对上述版本，添加回撤止盈；不过这里会有一个问题，因为加仓或者浮动止损的操作都是在有仓位的基础下进行的操作，故都在同一个循环中，因为可能同时满足加仓和止损的条件，于是采用if-elif的方式进行取舍；分别尝试先做止损判断和先做加仓的判断两个版本； 最大回撤都有大幅下降，先判断加仓的版本，虽然最大回撤稍大，但整体回报提升了2个百分点

加入止损后，sharpe和胜率有明显提高，但是盈亏比并没有明显的变化

波动率倍数止损效果比较难调

分别按照无止损，无方向-2倍 无方向-1.5倍 有方向-1.5倍 有方向-1.25倍

最大回撤有明显减小的版本是有方向的-1.25倍

故决定采用波动率标准差止损

波动率标准差的方式止损

貌似也不尽人意，怎么办呢

于是尝试过2倍和3倍标准差止损后，又尝试了2.5倍标准差，发现策略有了明显的改善，最大回撤降到了11%，年化收益破了7

仓位管理中，如果用凯利公式下单，是不是会更好点，那么在策略中，如何使用凯利公式下单呢